## An attempt to answer the question: "Which types of CO2 emission are most sensitve to increase in GDP?"

In [1]:
#import dependencies
import psycopg2
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

In [2]:
#define a function to connect to the database.  function code found on www.naysan.ca

param_dict = {
    "host":"ec2-3-211-167-220.compute-1.amazonaws.com",
    "database":"d89a2icfqufifv",
    "user":"hrcykodaiuwpab",
    "password":"92ba424e0b5a8c22538d55bc45ae0b239a9e1c23b25ec20d539d9e67afefa8be"
}

def connect(params_dic):
    #connect to the PostgreSQL database server
    conn = None
    try:
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params_dic)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1)
    print("Connection successfull")
    return conn

In [3]:
# define a function to import data from the database and convert it to a dataframe.  function code found on www.naysan.ca

def postgresql_to_dataframe(conn, select_query, column_names):
    
    #Tranforms a SELECT query into a pandas dataframe
    
    cursor = conn.cursor()
    try:
        cursor.execute(select_query)
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        cursor.close()
        return 1
    
    # Naturally we get a list of tupples
    tupples = cursor.fetchall()
    cursor.close()
    
    # We just need to turn it into a pandas dataframe
    df = pd.DataFrame(tupples, columns=column_names)
    return df

In [4]:
# execute function to connect to the database
conn = connect(param_dict)

Connecting to the PostgreSQL database...
Connection successfull


In [5]:
# execute function to import co2 data
column_names = ['iso_code','country','year','co2','co2_growth_prct','co2_growth_abs','consumption_co2','trade_co2','trade_co2_share','co2_per_capita','consumption_co2_per_capita','share_global_co2','cumulative_co2','share_global_cumulative_co2','co2_per_gdp','consumption_co2_per_gdp','co2_per_unit_energy','cement_co2','coal_co2','flaring_co2','gas_co2','oil_co2','cement_co2_per_capita','coal_co2_per_capita','flaring_co2_per_capita','gas_co2_per_capita','oil_co2_per_capita','total_ghg','ghg_per_capita','methane','methane_per_capita','nitrous_oxide','nitrous_oxide_per_capita','primary_energy_consumption','energy_per_capita','energy_per_gdp','population','gdp']

co2_df = postgresql_to_dataframe(conn, "select * from co2_data", column_names)
co2_df

,iso_code,country,year,co2,co2_growth_prct,co2_growth_abs,consumption_co2,trade_co2,trade_co2_share,co2_per_capita,...,ghg_per_capita,methane,methane_per_capita,nitrous_oxide,nitrous_oxide_per_capita,primary_energy_consumption,energy_per_capita,energy_per_gdp,population,gdp
0,AFG,Afghanistan,1949,0.015,NaN,NaN,NaN,NaN,NaN,0.002,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7663783.0,NaN
1,AFG,Afghanistan,1950,0.084,475.000,0.070,NaN,NaN,NaN,0.011,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7752000.0,1.949480e+10
2,AFG,Afghanistan,1951,0.092,8.696,0.007,NaN,NaN,NaN,0.012,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7840000.0,2.006385e+10
3,AFG,Afghanistan,1952,0.092,0.000,0.000,NaN,NaN,NaN,0.012,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7936000.0,2.074235e+10
4,AFG,Afghanistan,1953,0.106,16.000,0.015,NaN,NaN,NaN,0.013,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8040000.0,2.201546e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24011,ZWE,Zimbabwe,2014,11.962,2.838,0.330,12.760,0.798,6.671,0.880,...,4.865,11.24,0.827,6.27,0.461,NaN,NaN,NaN,13587000.0,2.474828e+10
24012,ZWE,Zimbabwe,2015,12.163,1.685,0.202,13.010,0.847,6.966,0.880,...,4.885,11.87,0.859,6.68,0.484,NaN,NaN,NaN,13815000.0,2.503057e+10
24013,ZWE,Zimbabwe,2016,10.807,-11.146,-1.356,11.809,1.001,9.264,0.770,...,4.703,11.92,0.850,6.55,0.467,NaN,NaN,NaN,14030000.0,2.515176e+10
24014,ZWE,Zimbabwe,2017,12.026,11.274,1.218,12.531,0.505,4.198,0.845,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14237000.0,NaN


In [6]:
# execute funtion to import temperature data
column_names2 = ['iso_code','country','year','Temperature']

temps_df = postgresql_to_dataframe(conn, "select * from temperature_data", column_names2)
temps_df

,iso_code,country,year,Temperature
0,AFG,Afghanistan,1901,13.172872
1,AFG,Afghanistan,1902,13.342649
2,AFG,Afghanistan,1903,12.006417
3,AFG,Afghanistan,1904,12.531729
4,AFG,Afghanistan,1905,12.267143
...,...,...,...,...
22731,ZWE,Zimbabwe,2012,21.910075
22732,ZWE,Zimbabwe,2013,21.624350
22733,ZWE,Zimbabwe,2014,21.710483
22734,ZWE,Zimbabwe,2015,22.327625


In [7]:
# remove entries from 'country' column that don't represent individual countries or regions
co2_countries = co2_df.loc[(co2_df['country'] != 'World') & (co2_df['country'] != 'Kuwaiti Oil Fires') & (co2_df['country'] != 'Antarctic Fisheries') & (co2_df['country'] != 'Statistical Difference')]

In [57]:
# create new dataframe with most pertinent columns for initial analysis
#country_data = co2_countries[['country','year','co2','trade_co2','cement_co2','coal_co2','flaring_co2','gas_co2','oil_co2','methane','nitrous_oxide','population','gdp']]
country_data = co2_countries[['country','year','oil_co2','population','gdp']]
country_data

,country,year,oil_co2,population,gdp
0,Afghanistan,1949,0.000,7663783.0,NaN
1,Afghanistan,1950,0.066,7752000.0,1.949480e+10
2,Afghanistan,1951,0.066,7840000.0,2.006385e+10
3,Afghanistan,1952,0.062,7936000.0,2.074235e+10
4,Afghanistan,1953,0.066,8040000.0,2.201546e+10
...,...,...,...,...,...
24011,Zimbabwe,2014,3.726,13587000.0,2.474828e+10
24012,Zimbabwe,2015,3.624,13815000.0,2.503057e+10
24013,Zimbabwe,2016,3.162,14030000.0,2.515176e+10
24014,Zimbabwe,2017,3.331,14237000.0,NaN


In [58]:
country_data.sort_values('year',inplace=True, ascending=False )
country_data

,country,year,oil_co2,population,gdp
24015,Zimbabwe,2018,3.427,14439000.0,NaN
2664,Belize,2018,0.571,383000.0,NaN
2792,Bermuda,2018,0.622,63000.0,NaN
16786,Pakistan,2018,71.401,212228000.0,NaN
18093,Russia,2018,411.892,145734000.0,NaN
...,...,...,...,...,...
7751,Europe (excl. EU-28),1751,NaN,NaN,NaN
1332,Asia (excl. China & India),1751,NaN,NaN,NaN
22625,United States,1751,NaN,NaN,NaN
15466,North America,1751,NaN,NaN,NaN


In [60]:
#drop NAs
clean_co2_df = country_data.dropna()
clean_co2_df

,country,year,oil_co2,population,gdp
21088,Taiwan,2016,70.374,23618000.0,9.850000e+11
2594,Belgium,2016,45.772,11354000.0,4.480000e+11
2402,Belarus,2016,19.212,9446000.0,1.800000e+11
14107,Montenegro,2016,0.832,627000.0,1.176738e+10
13499,Malta,2016,1.412,436000.0,1.212167e+10
...,...,...,...,...,...
8396,France,1820,0.000,31549988.0,4.506250e+10
17381,Poland,1820,0.000,10508375.0,6.860308e+09
22390,United Kingdom,1820,0.000,16221883.0,4.632226e+10
3825,Canada,1820,0.000,879432.0,1.260720e+09


In [61]:
#Total row count after drop na 
    #12848 
    
#over 10,000 rows 
    #oil_co2 12,773
    #cement_co2 11,131
    
#over 5,000-10,000 rows
    #coal_co2 9,995
    #gas_co2 8,877
    #flaring_co2 7,041
    

In [38]:
clean_co2_df['country'].unique().tolist()

['Taiwan',
 'Belgium',
 'Belarus',
 'Ireland',
 'Norway',
 'Romania',
 'Portugal',
 'Nicaragua',
 'Azerbaijan',
 'Bangladesh',
 'Barbados',
 'Croatia',
 'Angola',
 'Slovenia',
 'Qatar',
 'Poland',
 'Cyprus',
 'Czechia',
 'Indonesia',
 'Zambia',
 'Turkmenistan',
 'Luxembourg',
 'Slovakia',
 'Bahrain',
 'Iraq',
 'Iran',
 'South Africa',
 'Philippines',
 'Libya',
 'Kyrgyzstan',
 'Ecuador',
 'Uzbekistan',
 'Sweden',
 'Kuwait',
 'Dominican Republic',
 'Egypt',
 'Uruguay',
 'South Korea',
 'Laos',
 'Austria',
 'Venezuela',
 'Sri Lanka',
 'Liberia',
 'Spain',
 'Armenia',
 'Lebanon',
 'Oman',
 'Latvia',
 'Sudan',
 'El Salvador',
 'Kenya',
 'Afghanistan',
 'Jamaica',
 'Switzerland',
 'Lithuania',
 'Ukraine',
 'Mongolia',
 'Netherlands',
 'Italy',
 'Democratic Republic of Congo',
 'Uganda',
 'Peru',
 'Ethiopia',
 'Syria',
 'Israel',
 'Estonia',
 'Paraguay',
 'New Zealand',
 'Moldova',
 'Zimbabwe',
 'Djibouti',
 'Kazakhstan',
 'United Kingdom',
 'Jordan',
 'Denmark',
 'Japan',
 'North Macedonia',